In [ ]:
!pip install autokeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 KB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.8/466.8 KB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.3/167.3 KB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 99.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 81.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')

PATH = "/content/drive/MyDrive/ML-Olympiad/Detect-ChatGpt-answers/"

train = pd.read_csv(PATH+'data/train.csv')
test = pd.read_csv(PATH+'data/test.csv')
submission = pd.read_csv(PATH+'data/sample_submission.csv')

Mounted at /content/drive


## 1. 전처리

In [ ]:
train["text"]=train["prompt"]+train["answer"]
train=train[["text","AI"]]

In [ ]:
test["text"]=test["prompt"]+test["answer"]
test=test["text"]

In [ ]:
train["text"].isna().sum()

1

In [ ]:
train = train[train["text"].isna()==False]

In [ ]:
train["AI"].isna().sum()

1

In [ ]:
train = train[train["AI"].isna()==False]

In [ ]:
test.isna().sum()

0

In [ ]:
train

,text,AI
0,What is the future of AI?The future of Artific...,1.0
1,What is your biggest challenge in your career?...,0.0
2,What is the tallest mountain in the world?The ...,1.0
3,What are the best 5 super cars?McLaren 720S.\r...,0.0
4,What is deep learning?a type of machine learni...,0.0
...,...,...
743,What is the most important thing in a family?T...,0.0
744,What are the best machine learning Models?Line...,0.0
745,How can I chop onions without crying?The best ...,0.0
746,What is the best way to stay healthy?The best ...,0.0


In [ ]:
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 16.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=fba738eaffb1ced51378b58488cf3ddfae35a7c43527e49127a5def487ecab2d
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji


In [ ]:
import numpy as np

In [ ]:
import re
def removeHTML(x):
    html=re.compile(r'<.*?>')
    return html.sub(r'',x)
    
import emoji
def dataPreprocessing(x): 
    x = x.lower()
    x = removeHTML(x)
    x = emoji.demojize(x, delimiters=(" ", " "))
    x = re.sub("@\w+", '',x) # removing mentions (@)
    x = re.sub("'\d+", '',x)
    x = re.sub("\d+", '',x)
    x = re.sub(r"[^\w\s]", '',x) # to remove symbols
    x = re.sub("http\w+", '',x)
    x = re.sub("\s[a-z]\s", '',x)
    x = x.strip()
    return x

In [ ]:

train_text=train["text"].apply(lambda x: dataPreprocessing(x))

In [ ]:
test_text=test.apply(lambda x: dataPreprocessing(x))

In [ ]:
train_text

0      what is the future of aithe future of artifici...
1      what is your biggest challenge in your careerm...
2      what is the tallest mountain in the worldthe t...
3      what are the best  super carsmclaren\nferrari ...
4      what is deep learninga type of machine learnin...
                             ...                        
743    what is the most important thing infamilythe m...
744    what are the best machine learning modelslinea...
745    how canchop onions without cryingthe best thin...
746    what is the best way to stay healthythe best w...
747    do you think naruto is overratedi think naruto...
Name: text, Length: 746, dtype: object

In [ ]:
from sklearn.model_selection import train_test_split

X = train_text
y = train['AI']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
pd.DataFrame(X_train, columns=['text'])

,text
70,what is the name of the unit used to measure p...
164,what is the name of the largest underwater cav...
712,what is the name of the gas that makes up the ...
265,what is the name of the largest underwater cav...
250,what is the most important factor in makingdec...
...,...
71,what is your biggest strengthmy biggest streng...
106,what is the currency of australiaaustralian do...
270,what is the best way to stay positivethe best ...
435,what is the name of the largest underwater cav...


In [ ]:
pd.DataFrame(y_train, columns=['AI'])

,AI
70,1.0
164,0.0
712,0.0
265,0.0
250,0.0
...,...
71,0.0
106,0.0
270,0.0
435,0.0


In [ ]:
# train_data = pd.concat([pd.DataFrame(X_train, columns=['text']), pd.DataFrame(y_train, columns=['AI'])], axis=1)
# train_data

In [ ]:
# test_data = pd.concat([pd.DataFrame(X_test, columns=['text']), pd.DataFrame(y_test, columns=['AI'])], axis=1)
# test_data

# 2. 모델링

In [ ]:
import os

import numpy as np
import tensorflow as tf
from sklearn.datasets import load_files

import autokeras as ak

In [ ]:
# Initialize the text classifier.
clf = ak.TextClassifier(
    overwrite=True, max_trials=100, objective='val_accuracy', seed=32
)  

In [ ]:
# 모델 저장 경로
model_dir = PATH+"model/autokeras_trial100_230304/"
if not os.path.exists(model_dir):
    os.mkdir(model_dir)

# 체크포인트 파일 저장 경로
checkpoint_path = model_dir+"checkpoints_model_autokeras_230304.ckpt"

In [ ]:
callbacks = [
tf.keras.callbacks.ModelCheckpoint(# 이거 하고 다시 해보기(지금 코드는 적용안됨)
    checkpoint_path, save_weights_only=True, 
    monitor="val_accuracy", mode="max", verbose=1, save_best_only=True
),
tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_accuracy", factor=0.5, patience=8, min_lr=0.0001
),
tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", mode='max', patience=20, verbose=1), # val_acc로 설정했는데 모델마다 먹히는 경우가 다름 -> 버그로 추정?
]

In [ ]:
# Feed the text classifier with training data.
clf.fit(X_train.values, y_train.values, callbacks=callbacks, validation_split=0.2, epochs=50)

Trial 18 Complete [00h 06m 56s]
val_accuracy: 0.9404761791229248

Best val_accuracy So Far: 0.9404761791229248
Total elapsed time: 01h 21m 08s

Search: Running Trial #19

Value             |Best Value So Far |Hyperparameter
bert              |bert              |text_block_1/block_type
0                 |0                 |classification_head_1/dropout
adam_weight_decay |adam_weight_decay |optimizer
0.0001            |2e-05             |learning_rate
512               |512               |text_block_1/bert_block_1/max_sequence_length
5000              |20000             |text_block_1/max_tokens

Epoch 1/50
16/16 [==============================] - ETA: 0s - loss: 0.7265 - accuracy: 0.5117
Epoch 1: val_loss improved from inf to 0.48682, saving model to /content/drive/MyDrive/ML-Olympiad/Detect-ChatGpt-answers/model/autokeras_trial100_230304/checkpoints_model_autokeras_230304.ckpt
16/16 [==============================] - 72s 1s/step - loss: 0.7265 - accuracy: 0.5117 - val_loss: 0.4868 - val

KeyboardInterrupt: ignored

# 3. 성능 평가

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
# Predict with the best model.
predicted_y = clf.predict(X_test.values)
# Evaluate the best model with testing data.
print(classification_report(y_test.values, predicted_y))

In [ ]:
# model_to_use = predictor.get_model_best()
model_pred = clf.predict(test_text.values)

In [ ]:
submission['Category'] = model_pred

In [ ]:
submission = submission.astype({'Category':'int64'})

In [ ]:
submission

In [ ]:
submission.to_csv(PATH+'submit/autokeras_text_trial100_submission_230304.csv', index=False)